<a href="https://colab.research.google.com/github/snbigft/API_web_noQuery/blob/main/API_web_noQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn psycopg2-binary nest-asyncio pyngrok python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
import os
import json
import psycopg2
import uvicorn
import nest_asyncio
import requests
from fastapi import FastAPI, Request , Query , HTTPException
from dotenv import load_dotenv
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv
from pyngrok import ngrok
from datetime import datetime


# Cargar variables de entorno desde .env
load_dotenv("/content/drive/My Drive/Colab Notebooks/.env")

# URL del servicio externo
LOTTO_API_URL = "https://www.lotto-italia.it/gdl/estrazioni-e-vincite/estrazioni-del-lotto.json"

# Obtener credenciales de la base de datos desde variables de entorno
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")

# Aplicar nest_asyncio para evitar conflictos de bucle de eventos en Colab
nest_asyncio.apply()

# Configurar ngrok con tu authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


# Opciones válidas para los endpoints
RUOTA_OPTIONS = ["Bari", "Cagliari", "Firenze", "Genova", "Milano", "Napoli", "Palermo", "Roma", "Torino", "Venezia", "Nazionale"]
TIPO_OPTIONS = ["top 10 ritardatari", "top 10 frequenti", "ambo piú frequente", "top 10 ambi"]

app = FastAPI()

def get_ip_address(request: Request) -> str:
    return request.client.host

def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

def format_date(date_str):
    return datetime.strptime(date_str, "%Y%m%d").strftime("%d/%m/%Y")

@app.get("/absolute_statistics")
def get_absolute_statistics(request: Request, ruota: str = Query(None, enum=RUOTA_OPTIONS), tipo: str = Query(..., enum=TIPO_OPTIONS)):
    ip_address = get_ip_address(request)
    json_input = {
        "api": "absolute_statistics",
        "ip_address_caller": ip_address,
        "query": {
            "ruota": ruota,
            "tipo": tipo
        }
    }

    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.callproc("GET_ABSOLUTE_STATISTICS", [json.dumps(json_input)])
        response_code, response_body = cursor.fetchone()
        cursor.close()
        conn.close()

        # Convertir la respuesta de texto (que es un JSON en formato string) a un diccionario de Python
        response_body_dict = json.loads(response_body)

          # Devolver el código de respuesta y el cuerpo de la respuesta como JSON
        return {"response_code": response_code, "response_body": response_body_dict}

    except Exception as e:
        return {"error": str(e)}


# Endpoint para number_info
@app.get("/number_info")
def number_info(request: Request, ruota: str = Query(None, enum=RUOTA_OPTIONS), numero: int = Query(..., ge=1, le=90)):
  ip_address = get_ip_address(request)
  json_input = {
        "api": "number_info",
        "ip_address_caller": ip_address,
        "query": {
            "ruota": ruota,
            "numero": numero
        }
  }
  try:
    conn = get_db_connection()
    cursor = conn.cursor()

    cursor.callproc("GET_NUMBER_INFO", [json.dumps(json_input)])
    response_code, response_body = cursor.fetchone()
    cursor.close()
    conn.close()

    # Convertir la respuesta de texto (que es un JSON en formato string) a un diccionario de Python
    response_body_dict = json.loads(response_body)

    # Devolver el código de respuesta y el cuerpo de la respuesta como JSON
    return {"response_code": response_code, "response_body": response_body_dict}

  except Exception as e:
        return {"error": str(e)}

# Endpoint para ambo_info
@app.get("/ambo_info")
def ambo_info(request: Request, ruota: str = Query(None, enum=RUOTA_OPTIONS), numero_1: int = Query(..., ge=1, le=90), numero_2: int = Query(..., ge=1, le=90)):
  ip_address = get_ip_address(request)
  json_input = {
        "api": "ambo_info",
        "ip_address_caller": ip_address,
        "query": {
            "ruota": ruota,
            "numero_1": numero_1,
            "numero_2": numero_2
        }
  }
  try:
    conn = get_db_connection()
    cursor = conn.cursor()

    cursor.callproc("GET_AMBO_INFO", [json.dumps(json_input)])
    response_code, response_body = cursor.fetchone()
    cursor.close()
    conn.close()

    # Convertir la respuesta de texto (que es un JSON en formato string) a un diccionario de Python
    response_body_dict = json.loads(response_body)

    # Devolver el código de respuesta y el cuerpo de la respuesta como JSON
    return {"response_code": response_code, "response_body": response_body_dict}

  except Exception as e:
        return {"error": str(e)}

@app.post("/insert_lotto_data")
async def insert_lotto_data(payload: dict):
    try:
        # Obtener la fecha desde el JSON recibido
        date_str = payload.get("data")
        if not date_str:
            raise HTTPException(status_code=400, detail="El campo 'data' es obligatorio.")

        # Convertir la fecha al formato DD/MM/YYYY
        giorno = f"{date_str[6:8]}/{date_str[4:6]}/{date_str[0:4]}"
        date_sort = int(date_str)

        # Hacer la petición al servicio externo
        response = requests.post(LOTTO_API_URL, json={"data": date_str})
        if response.status_code != 200:
            raise HTTPException(status_code=500, detail="Error en la API externa de lotto-italia.it")

        lotto_data = response.json()

        # Verificar si la respuesta es válida
        if lotto_data.get("esito") != "OK":
            raise HTTPException(status_code=500, detail="La API de lotto-italia.it devolvió un error.")

        # Procesar los datos de la extracción
        conn = get_db_connection()
        cursor = conn.cursor()

        for estrazione in lotto_data["estrazione"]:
            ruota = estrazione["ruotaExtended"].capitalize()  # Convertir a formato "Bari" en vez de "BARI"

            for posicion, numero in enumerate(estrazione["numeri"], start=1):
                query = """
                    INSERT INTO lotto_it (giorno, ruota, numero, json_data, posicion, date_sort)
                    VALUES (%s, %s, %s, %s, %s, %s);
                """
                cursor.execute(query, (giorno, ruota, numero, None, posicion, date_sort))

        conn.commit()
        cursor.close()
        conn.close()

        # Devolver el JSON original recibido
        return {"message": "Datos insertados correctamente", "lotto_response": lotto_data}

    except Exception as e:
        return {"error": str(e)}

# Exponer la API con ngrok en Google Colab
public_url = ngrok.connect(8000).public_url
print(f"🔥 API en ejecución en: {public_url}/docs")

# Iniciar el servidor uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-154' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔥 API en ejecución en: https://b920-35-231-156-236.ngrok-free.app/docs


INFO:     Started server process [262]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     139.47.127.5:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "POST /insert_lotto_data HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "POST /insert_lotto_data HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "POST /insert_lotto_data HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "POST /insert_lotto_data HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "GET /ambo_info?ruota=Genova&numero_1=72&numero_2=31 HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "GET /ambo_info?ruota=Genova&numero_1=5&numero_2=58 HTTP/1.1" 200 OK
INFO:     139.47.127.5:0 - "GET /ambo_info?numero_1=48&numero_2=81 HTTP/1.1" 200 OK
